# Clustering and Topic Modeling of Student Questions

This notebook demonstrates how to cluster and explore **student questions** using embeddings, dimensionality reduction, clustering, and topic modeling.  

We will follow these steps:

1. Convert questions into embedding vectors  
2. Reduce the embeddings to 5 dimensions  
3. Cluster the reduced embeddings  
4. Generate keyword-based topics  
5. Reduce embeddings to 2 dimensions for visualization  
6. Visualize questions + topics  
7. Use OpenAI to generate short topic labels

In [ ]:
from google.colab import files

# Upload the local dataset file (choose the latest extracted_user_inputs_<date>.txt)
uploaded = files.upload()

# Get the uploaded file name
filename = list(uploaded.keys())[0]

# Load questions from the uploaded file
with open(filename, "r", encoding="utf-8") as f:
    questions = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(questions)} questions")
print("Sample:", questions[:5])


## Create Embeddings
We will use a **sentence transformer** to embed each question into a high-dimensional vector

In [ ]:
from openai import OpenAI
import numpy as np
from google.colab import userdata
from tqdm.auto import tqdm

# Use Colab's secrets manager for API key
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

# Function to generate embeddings in batches
def generate_embeddings_in_batches(client, questions, model="text-embedding-3-small", batch_size=1000):
    embeddings = []
    for i in tqdm(range(0, len(questions), batch_size)):
        batch = questions[i : i + batch_size]
        try:
            response = client.embeddings.create(
                model=model,
                input=batch
            )
            embeddings.extend([d.embedding for d in response.data])
        except Exception as e:
            print(f"Error processing batch {i//batch_size}: {e}")
            # Depending on the error, you might want to implement retry logic or skip the batch
            continue
    return np.array(embeddings)

# Generate embeddings with OpenAI in batches
embeddings = generate_embeddings_in_batches(client, questions)

# Check the dimensions of the resulting embeddings
embeddings.shape

## Reduce embeddings to 5 dimensions (for clustering)

In [ ]:
from umap import UMAP

# Reduce from 1536 → 5 dims
umap_model = UMAP(
    n_components=5,
    min_dist=0.0,
    metric='cosine',
    random_state=42
)
reduced_embeddings = umap_model.fit_transform(embeddings)

## Cluster the reduced embeddings

In [ ]:
from hdbscan import HDBSCAN
import numpy as np

# Cluster with HDBSCAN
hdbscan_model = HDBSCAN(
    min_cluster_size=15, # tweak if clusters are too small/too many
    metric="euclidean",
    cluster_selection_method="eom"
)
clusters = hdbscan_model.fit_predict(reduced_embeddings)

# How many clusters were found?
unique_clusters, counts = np.unique(clusters, return_counts=True)
print(f"Number of clusters found: {len(unique_clusters)}")

# Calculate and print the number of clustered and unclustered questions
unclustered_count = counts[unique_clusters == -1][0] if -1 in unique_clusters else 0
clustered_count = len(clusters) - unclustered_count

print(f"Number of questions clustered: {clustered_count}")
print(f"Number of questions not clustered: {unclustered_count}")

In [ ]:
import pandas as pd

# Reduce again for visualization (1536 → 2)
reduced_2d = UMAP(
    n_components=2,
    min_dist=0.0,
    metric='cosine',
    random_state=42
).fit_transform(embeddings)

# Create dataframe
df = pd.DataFrame(reduced_2d, columns=["x", "y"])
df["questions"] = questions
df["cluster"] = clusters

In [ ]:
from bertopic import BERTopic

# Train our model with our previously defined models
topic_model = BERTopic(
    embedding_model=None, # We already have embeddings
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True
)

topics, probs = topic_model.fit_transform(questions, embeddings)

topic_model.get_topic_info().head(10)

In [ ]:
from bertopic.representation import OpenAI

prompt = """
I have a topic that contains the following missionary questions:
[DOCUMENTS]

The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short topic label in the format:
topic: <short topic label>
"""

representation_model = OpenAI(
    client,
    model="gpt-4o", # Corrected model name
    exponential_backoff=True,
    chat=True,
    prompt=prompt
)

topic_model.update_topics(questions, representation_model=representation_model)
topic_model.get_topic_info().head(10)

## Visualizations

In [ ]:
# Interactive doc visualization
fig = topic_model.visualize_documents(
    questions,
    reduced_embeddings=reduced_2d,
    width=1200,
    hide_annotations=True
)
fig.show()

# Keyword barchart
topic_model.visualize_barchart()

# Heatmap of topics
topic_model.visualize_heatmap

# Hierarchical topic structure
topic_model.visualize_hierarchy()

In [ ]:
# Get topic information
# This calls the get_topic_info() method on the topic_model object
# This method returns a pandas DataFrame containing detailed information about each topic
topic_info = topic_model.get_topic_info()

# Display the topic information
# This uses the display() function to show the contents of the topic_info DataFrame
# display() is often used in environments like Colab to render DataFrames in a more readable, formatted way
display(topic_info)

## Create Question-Topic Dataset (Using Representations)

Now we'll create a dataset that maps each question to its cluster-generated topic representation. This will be our "ground truth" for testing OpenAI's classification accuracy.

In [ ]:
# Create a mapping from cluster ID to topic representation (not name)
# Note: representations are stored as lists, so we take the first element
topic_map = topic_info.set_index("Topic")["Representation"].to_dict()

# Create a function to get the first representation from the list
def get_topic_representation(cluster_id):
    if cluster_id in topic_map:
        representation = topic_map[cluster_id]
        # Representation is stored as a list, get the first element
        if isinstance(representation, list) and len(representation) > 0:
            return representation[0]
        else:
            return str(representation)
    else:
        return "Other"  # For unclustered questions (-1)

# Create DataFrame with questions and their cluster-assigned topic representations
df_questions_topics = pd.DataFrame({
    'question': questions,
    'cluster': topics,  # This comes from topic_model.fit_transform
    'cluster_topic': [get_topic_representation(cluster_id) for cluster_id in topics]
})

# Remove questions that were not clustered (cluster = -1)
df_questions_topics = df_questions_topics[df_questions_topics['cluster'] != -1]

print(f"Total questions with assigned topics: {len(df_questions_topics)}")
print(f"Number of unique topics: {len(df_questions_topics['cluster_topic'].unique())}")
print("\nSample data:")
display(df_questions_topics.head(10))

## OpenAI Classification Testing Results

The comprehensive model testing above shows the performance of different OpenAI models on the classification task. The testing uses an enhanced approach that forces models to select from available topics rather than defaulting to "Other".

**Testing Summary**: 
- ✅ **Model Comparison**: All available OpenAI models tested on {MODEL_COMPARISON_SAMPLE_SIZE} questions
- ✅ **Best Model Testing**: Detailed test with {BEST_MODEL_TEST_SIZE} questions using the top performer  
- ✅ **Full Dataset**: Optional testing on all {len(df_questions_topics)} questions (configurable)

**Goal**: Achieve 90%+ accuracy to justify replacing clustering with direct OpenAI classification.

## Configuration Settings

Set up testing parameters and model configurations for easy customization.

In [ ]:
# ====================================================================
# TESTING CONFIGURATION - Change these variables to customize testing
# ====================================================================

# Number of questions for initial model comparison (smaller sample)
MODEL_COMPARISON_SAMPLE_SIZE = 20

# Number of questions for best model testing (medium sample)
BEST_MODEL_TEST_SIZE = 100

# Whether to run full dataset test (all questions) - Set to True/False
RUN_FULL_DATASET_TEST = False  # Change to True to test all questions

# Concurrency settings for API calls
OPTIMAL_CONCURRENCY = 16  # Recommended for stability
HIGH_CONCURRENCY = 32     # For faster processing (may hit rate limits)

print(f"📋 TESTING CONFIGURATION:")
print(f"   Model comparison sample: {MODEL_COMPARISON_SAMPLE_SIZE} questions")
print(f"   Best model test: {BEST_MODEL_TEST_SIZE} questions") 
print(f"   Full dataset test: {'ENABLED' if RUN_FULL_DATASET_TEST else 'DISABLED'}")
print(f"   Concurrency setting: {OPTIMAL_CONCURRENCY} parallel requests")
print("="*60)

In [ ]:
# ====================================================================
# OPENAI MODEL DATABASE - Complete model specifications and pricing
# ====================================================================

# Model specifications with accurate pricing (as of December 2025)
OPENAI_MODELS = {
    # GPT-5 Series (Latest - Released August 2025)
    "gpt-5": {
        "name": "GPT-5",
        "description": "Our smartest, fastest, most useful model yet",
        "input_price": 1.25,      # $1.25 per 1M tokens
        "output_price": 10.00,    # $10.00 per 1M tokens
        "context_window": 400000,
        "api_params": {"max_completion_tokens": True, "temperature_required": False}
    },
    "gpt-5-mini": {
        "name": "GPT-5 Mini", 
        "description": "Faster, cheaper version of GPT-5 for well-defined tasks",
        "input_price": 0.25,      # $0.25 per 1M tokens
        "output_price": 2.00,     # $2.00 per 1M tokens
        "context_window": 400000,
        "api_params": {"max_completion_tokens": True, "temperature_required": False}
    },
    "gpt-5-nano": {
        "name": "GPT-5 Nano",
        "description": "Fastest, cheapest GPT-5 - optimized for classification",
        "input_price": 0.05,      # $0.05 per 1M tokens
        "output_price": 0.40,     # $0.40 per 1M tokens  
        "context_window": 400000,
        "api_params": {"max_completion_tokens": True, "temperature_required": False}
    },
    
    # GPT-4o Series (Current generation)
    "gpt-4o": {
        "name": "GPT-4o",
        "description": "High-performance multimodal model",
        "input_price": 2.50,      # Estimated pricing
        "output_price": 10.00,    # Estimated pricing
        "context_window": 128000,
        "api_params": {"max_completion_tokens": False, "temperature_required": True}
    },
    "gpt-4o-mini": {
        "name": "GPT-4o Mini",
        "description": "Efficient and fast, cost-effective option",
        "input_price": 0.15,      # $0.15 per 1M tokens
        "output_price": 0.60,     # $0.60 per 1M tokens
        "context_window": 128000,
        "api_params": {"max_completion_tokens": False, "temperature_required": True}
    },
    "gpt-4o-2024-08-06": {
        "name": "GPT-4o (Aug 2024)",
        "description": "Specific snapshot version of GPT-4o",
        "input_price": 2.50,      # Estimated pricing
        "output_price": 10.00,    # Estimated pricing  
        "context_window": 128000,
        "api_params": {"max_completion_tokens": False, "temperature_required": True}
    },
    
    # GPT-4 Series (Previous generation)
    "gpt-4-turbo": {
        "name": "GPT-4 Turbo",
        "description": "Previous generation flagship model",
        "input_price": 10.00,     # $10.00 per 1M tokens
        "output_price": 30.00,    # $30.00 per 1M tokens
        "context_window": 128000,
        "api_params": {"max_completion_tokens": False, "temperature_required": True}
    },
    "gpt-4": {
        "name": "GPT-4",
        "description": "Original GPT-4 model",
        "input_price": 30.00,     # $30.00 per 1M tokens
        "output_price": 60.00,    # $60.00 per 1M tokens
        "context_window": 8192,
        "api_params": {"max_completion_tokens": False, "temperature_required": True}
    },
    
    # GPT-3.5 Series (Older generation)
    "gpt-3.5-turbo": {
        "name": "GPT-3.5 Turbo", 
        "description": "Fast and affordable legacy model",
        "input_price": 0.50,      # $0.50 per 1M tokens
        "output_price": 1.50,     # $1.50 per 1M tokens
        "context_window": 16385,
        "api_params": {"max_completion_tokens": False, "temperature_required": True}
    },
    "gpt-3.5-turbo-0125": {
        "name": "GPT-3.5 Turbo (Jan 2024)",
        "description": "Specific snapshot of GPT-3.5 Turbo",
        "input_price": 0.50,      # $0.50 per 1M tokens
        "output_price": 1.50,     # $1.50 per 1M tokens
        "context_window": 16385, 
        "api_params": {"max_completion_tokens": False, "temperature_required": True}
    }
}

# Print model database
print("🔍 OPENAI MODEL DATABASE")
print("="*80)
for model_id, info in OPENAI_MODELS.items():
    print(f"📋 {info['name']} ({model_id})")
    print(f"   💰 Pricing: ${info['input_price']:.2f} input / ${info['output_price']:.2f} output per 1M tokens")
    print(f"   📝 Description: {info['description']}")
    print(f"   📊 Context: {info['context_window']:,} tokens")
    print()

print(f"✅ Total models configured: {len(OPENAI_MODELS)}")
print("="*80)

In [ ]:
# ====================================================================
# INTELLIGENT MODEL CONFIGURATION SYSTEM
# ====================================================================

def get_model_config(model_name: str) -> dict:
    """
    Get the appropriate API configuration for different OpenAI models.
    Handles differences between GPT-5 and older models.
    """
    if model_name in OPENAI_MODELS:
        model_info = OPENAI_MODELS[model_name]
        api_params = model_info["api_params"]
        
        config = {
            "model": model_name,
            "pricing": {
                "input": model_info["input_price"],
                "output": model_info["output_price"]
            },
            "context_window": model_info["context_window"],
            "name": model_info["name"]
        }
        
        # Configure API parameters based on model type
        if api_params["max_completion_tokens"]:
            # GPT-5 series uses max_completion_tokens
            config["max_tokens_param"] = "max_completion_tokens"
            config["max_tokens_value"] = 200  # Sufficient for topic names without truncation
            # FIX: GPT-5 models work better with explicit temperature
            config["temperature"] = 0.1  # Low but consistent temperature for all models
        else:
            # GPT-4 and older use max_tokens
            config["max_tokens_param"] = "max_tokens"
            config["max_tokens_value"] = 200  # Increased from 100 for better responses
            config["temperature"] = 0.1  # Consistent temperature across all models
            
        return config
    else:
        # Default configuration for unknown models
        return {
            "model": model_name,
            "max_tokens_param": "max_tokens",
            "max_tokens_value": 100,
            "temperature": 0,
            "pricing": {"input": 0.0, "output": 0.0},
            "context_window": 8192,
            "name": model_name
        }

def estimate_cost(input_tokens: int, output_tokens: int, model_name: str) -> float:
    """Calculate estimated cost for API calls"""
    if model_name in OPENAI_MODELS:
        info = OPENAI_MODELS[model_name]
        input_cost = (input_tokens / 1_000_000) * info["input_price"]
        output_cost = (output_tokens / 1_000_000) * info["output_price"]
        return input_cost + output_cost
    return 0.0

# Test the configuration system
print("🔧 TESTING MODEL CONFIGURATION SYSTEM")
print("="*60)

test_models = ["gpt-5-nano", "gpt-4o-mini", "gpt-3.5-turbo"]
for model in test_models:
    config = get_model_config(model)
    print(f"📋 {config['name']}:")
    print(f"   🔧 API param: {config['max_tokens_param']} = {config['max_tokens_value']}")
    print(f"   🌡️ Temperature: {config['temperature']}")
    print(f"   💰 Pricing: ${config['pricing']['input']:.2f}/${config['pricing']['output']:.2f} per 1M tokens")
    
    # Estimate cost for 100 questions (rough estimate)
    est_cost = estimate_cost(50000, 5000, model)  # ~500 tokens per question
    print(f"   💸 Est. cost for 100 questions: ${est_cost:.4f}")
    print()

print("✅ Configuration system ready!")
print("="*60)

In [ ]:
# ====================================================================
# COMPREHENSIVE MODEL TESTING - All OpenAI Models
# ====================================================================

async def classify_with_forced_selection(question: str, available_topics: List[str], model_config: dict) -> str:
    """
    Enhanced classification that FORCES the model to choose from available topics.
    No 'Other' option - must pick the best matching topic.
    """
    # Create numbered topic list for clarity
    numbered_topics = []
    for i, topic in enumerate(available_topics, 1):
        numbered_topics.append(f"{i}. {topic}")
    
    topics_str = "\n".join(numbered_topics)
    
    # Enhanced prompt with clearer instructions
    prompt = f"""CLASSIFICATION TASK: You must select exactly ONE topic from the list below that best matches the student question.

AVAILABLE TOPICS (select one):
{topics_str}

CRITICAL INSTRUCTIONS:
- You MUST choose one of the numbered topics above
- Respond with ONLY the exact topic name (without the number)
- Do NOT say "Other", "None", or refuse to answer
- If uncertain, pick the closest related topic
- Look for key concepts: registration, courses, portal access, scholarships, certificates

STUDENT QUESTION: "{question}"

RESPONSE (topic name only):"""

    try:
        # Build API call with model-specific parameters
        api_params = {
            "model": model_config["model"],
            "messages": [
                {"role": "system", "content": "You are a classification expert. Always select from the provided topics. Never refuse or say 'Other'."},
                {"role": "user", "content": prompt}
            ]
        }
        
        # Add model-specific parameters
        if model_config["max_tokens_param"] == "max_completion_tokens":
            api_params["max_completion_tokens"] = model_config["max_tokens_value"]
        else:
            api_params["max_tokens"] = model_config["max_tokens_value"]
            
        if model_config["temperature"] is not None:
            api_params["temperature"] = model_config["temperature"]
            
        response = await async_client.chat.completions.create(**api_params)
        result = response.choices[0].message.content.strip()
        
        # Optional debug logging (comment out for production)
        # print(f"🔍 Debug: '{question[:30]}...' -> Raw: '{result}' (len={len(result)})")
        
        # Clean up response more thoroughly
        result_clean = result.replace("•", "").replace("-", "").strip()
        result_clean = result_clean.rstrip('.').rstrip(':').rstrip(',')
        
        # Remove quotes and extra whitespace
        result_clean = result_clean.strip('"').strip("'").strip()
        
        # Remove any leading numbers (e.g., "1. Topic" -> "Topic")
        import re
        result_clean = re.sub(r'^\d+\.\s*', '', result_clean)
        result_clean = re.sub(r'^\d+\)\s*', '', result_clean)  # Handle "1) Topic" format
        
        # Handle empty responses immediately
        if not result_clean or len(result_clean) < 3:
            print(f"⚠️  Empty response for: '{question[:50]}...'")
            return available_topics[0]
        
        # Direct exact match first (case-insensitive)
        for topic in available_topics:
            if result_clean.lower() == topic.lower():
                return topic
        
        # Enhanced fuzzy matching with multiple strategies
        best_match = None
        max_score = 0
        
        for topic in available_topics:
            topic_lower = topic.lower()
            result_lower = result_clean.lower()
            
            # Strategy 1: Substring matching (both directions)
            if topic_lower in result_lower or result_lower in topic_lower:
                if len(result_clean) > 5:  # Avoid matching very short strings
                    return topic
            
            # Strategy 2: Word-by-word matching
            topic_words = set(topic_lower.split())
            result_words = set(result_lower.split())
            common_words = topic_words.intersection(result_words)
            
            if common_words:
                score = len(common_words) / max(len(topic_words), len(result_words))
                if score > max_score and score > 0.3:  # At least 30% word overlap
                    max_score = score
                    best_match = topic
        
        if best_match:
            return best_match
                
        # Enhanced keyword-based fallback
        question_lower = question.lower()
        result_lower = result_clean.lower()
        
        # Extended keyword matching
        keyword_groups = {
            "registration": ["registration", "register", "enroll", "signup"],
            "course": ["course", "class", "lesson", "curriculum"],
            "portal": ["portal", "website", "login", "access", "dashboard"],
            "scholarship": ["scholarship", "financial", "aid", "funding", "money"],
            "certificate": ["certificate", "diploma", "credential", "completion"],
            "technical": ["technical", "tech", "computer", "software", "online"],
            "pathway": ["pathway", "program", "byu"]
        }
        
        # Score topics based on keyword relevance
        topic_scores = {}
        for topic in available_topics:
            topic_lower = topic.lower()
            score = 0
            
            # Check for keyword matches
            for category, keywords in keyword_groups.items():
                for keyword in keywords:
                    if keyword in question_lower and keyword in topic_lower:
                        score += 2  # Strong match
                    elif keyword in question_lower or keyword in result_lower:
                        if keyword in topic_lower:
                            score += 1  # Partial match
            
            if score > 0:
                topic_scores[topic] = score
        
        # Return highest scoring topic if any keywords matched
        if topic_scores:
            best_topic = max(topic_scores.keys(), key=lambda t: topic_scores[t])
            print(f"⚡ Keyword fallback: '{question[:30]}...' -> '{best_topic}' (score: {topic_scores[best_topic]})")
            return best_topic
        
        # Final fallback with better logging
        print(f"⚠️  Final fallback used for: '{question[:50]}...'")
        print(f"   Raw response: '{result}'")
        print(f"   Cleaned: '{result_clean}'")
        print(f"   -> Defaulting to: '{available_topics[0]}'")
        return available_topics[0]
        
    except Exception as e:
        error_msg = str(e)
        print(f"❌ API Error classifying question: {error_msg}")
        print(f"   Question: '{question[:50]}...'")
        print(f"   Model: {model_config.get('model', 'unknown')}")
        
        # Check for specific API errors
        if "max_completion_tokens" in error_msg and model_config["max_tokens_param"] == "max_completion_tokens":
            print(f"   🔧 Hint: GPT-5 model parameter issue detected")
        elif "max_tokens" in error_msg and model_config["max_tokens_param"] == "max_tokens":
            print(f"   🔧 Hint: GPT-4 model parameter issue detected")
        elif "temperature" in error_msg:
            print(f"   🔧 Hint: Temperature parameter issue detected")
        
        return available_topics[0]  # Return first topic instead of "Other"

async def test_all_models_comprehensive():
    """Test all available OpenAI models with comprehensive comparison"""
    
    print(f"🚀 COMPREHENSIVE MODEL TESTING")
    print("="*80)
    print(f"📊 Testing {MODEL_COMPARISON_SAMPLE_SIZE} questions across all models")
    print(f"🎯 Using forced selection (no 'Other' allowed)")
    print("="*80)
    
    # Get sample data
    test_sample = df_questions_topics.head(MODEL_COMPARISON_SAMPLE_SIZE).to_dict('records')
    
    # Models to test (prioritized order)
    priority_models = [
        "gpt-5-nano",      # Best for classification
        "gpt-5-mini",      # Fast GPT-5
        "gpt-5",           # Full GPT-5
        "gpt-4o-mini",     # Current best baseline
        "gpt-4o",          # Current flagship
        "gpt-4-turbo",     # Previous generation
        "gpt-3.5-turbo",   # Legacy but fast
    ]
    
    model_results = {}
    
    for model_name in priority_models:
        model_config = get_model_config(model_name)
        print(f"\n🔬 Testing {model_config['name']} ({model_name})")
        print(f"   💰 Cost: ${model_config['pricing']['input']:.2f}/${model_config['pricing']['output']:.2f} per 1M tokens")
        
        # Test model availability first
        try:
            test_params = {
                "model": model_name,
                "messages": [{"role": "user", "content": "Test"}]
            }
            if model_config["temperature"] is not None:
                test_params["temperature"] = model_config["temperature"]
                
            test_response = await async_client.chat.completions.create(**test_params)
            print(f"   ✅ Model available")
        except Exception as e:
            print(f"   ❌ Model unavailable: {str(e)[:100]}...")
            model_results[model_name] = {"error": str(e), "available": False}
            continue
        
        # Run classification test
        start_time = time.time()
        sem = asyncio.Semaphore(8)  # Conservative concurrency for testing
        
        async def test_worker(question_data):
            async with sem:
                question = question_data['question']
                correct_topic = question_data['cluster_topic']
                
                predicted_topic = await classify_with_forced_selection(question, topics_list, model_config)
                
                return {
                    'question': question,
                    'correct_topic': correct_topic,
                    'predicted_topic': predicted_topic,
                    'is_correct': predicted_topic == correct_topic
                }
        
        try:
            tasks = [test_worker(q_data) for q_data in test_sample]
            results = await asyncio.gather(*tasks)
            end_time = time.time()
            
            # Calculate metrics
            correct = sum(1 for r in results if r['is_correct'])
            accuracy = (correct / len(results)) * 100
            processing_time = end_time - start_time
            
            # Estimate cost
            avg_input_tokens = 400  # Estimated tokens per prompt
            avg_output_tokens = 20  # Estimated tokens per response
            total_input = avg_input_tokens * len(results)
            total_output = avg_output_tokens * len(results)
            estimated_cost = estimate_cost(total_input, total_output, model_name)
            
            model_results[model_name] = {
                "available": True,
                "accuracy": accuracy,
                "correct": correct,
                "total": len(results),
                "processing_time": processing_time,
                "estimated_cost": estimated_cost,
                "config": model_config
            }
            
            print(f"   🎯 Accuracy: {accuracy:.1f}% ({correct}/{len(results)})")
            print(f"   ⚡ Time: {processing_time:.1f}s")
            print(f"   💸 Est. cost: ${estimated_cost:.6f}")
            
        except Exception as e:
            print(f"   ❌ Testing failed: {str(e)[:100]}...")
            model_results[model_name] = {"error": str(e), "available": True, "test_failed": True}
    
    return model_results

# Run comprehensive testing
comprehensive_results = await test_all_models_comprehensive()

In [ ]:
# ====================================================================
# RESULTS ANALYSIS AND BEST MODEL SELECTION
# ====================================================================

def analyze_comprehensive_results(results: dict):
    """Analyze and display comprehensive testing results"""
    
    print(f"\n🏆 COMPREHENSIVE MODEL COMPARISON RESULTS")
    print("="*90)
    
    # Separate available and unavailable models
    available_models = {k: v for k, v in results.items() if v.get("available", False) and not v.get("test_failed", False)}
    unavailable_models = {k: v for k, v in results.items() if not v.get("available", False) or v.get("test_failed", False)}
    
    if available_models:
        print(f"✅ SUCCESSFUL TESTS ({len(available_models)} models):")
        print("-" * 90)
        print(f"{'Model':<25} {'Accuracy':<12} {'Time':<8} {'Cost':<12} {'Notes':<20}")
        print("-" * 90)
        
        # Sort by accuracy (best first)
        sorted_models = sorted(available_models.items(), key=lambda x: x[1]['accuracy'], reverse=True)
        
        best_model = None
        best_accuracy = 0
        
        for model_name, result in sorted_models:
            accuracy = result['accuracy']
            time_taken = result['processing_time']
            cost = result['estimated_cost']
            
            # Determine best model
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model_name
            
            # Performance indicators
            if accuracy >= 80:
                perf_indicator = "🔥 EXCELLENT"
            elif accuracy >= 60:
                perf_indicator = "🌟 VERY GOOD"
            elif accuracy >= 40:
                perf_indicator = "⚡ GOOD"
            elif accuracy >= 20:
                perf_indicator = "📈 FAIR"
            else:
                perf_indicator = "🔄 NEEDS WORK"
            
            print(f"{model_name:<25} {accuracy:>6.1f}% {time_taken:>6.1f}s ${cost:>8.6f} {perf_indicator}")
        
        print("-" * 90)
        
        if best_model:
            best_result = available_models[best_model]
            best_config = best_result['config']
            
            print(f"\n🏆 BEST PERFORMING MODEL: {best_config['name']}")
            print(f"   📊 Model ID: {best_model}")
            print(f"   🎯 Accuracy: {best_accuracy:.1f}% ({best_result['correct']}/{best_result['total']})")
            print(f"   ⚡ Processing time: {best_result['processing_time']:.1f} seconds")
            print(f"   💰 Estimated cost: ${best_result['estimated_cost']:.6f}")
            print(f"   💸 Pricing: ${best_config['pricing']['input']:.2f}/${best_config['pricing']['output']:.2f} per 1M tokens")
            
            # Cost projection for larger tests
            cost_100 = estimate_cost(40000, 2000, best_model)  # ~400 tokens per question
            cost_full = estimate_cost(400000 * len(df_questions_topics) // 100, 20000 * len(df_questions_topics) // 100, best_model)
            
            print(f"\n📈 COST PROJECTIONS:")
            print(f"   💵 {BEST_MODEL_TEST_SIZE} questions: ~${cost_100:.4f}")
            print(f"   💵 All {len(df_questions_topics)} questions: ~${cost_full:.2f}")
            
            # Set global best model variable
            global GLOBAL_BEST_MODEL, GLOBAL_BEST_CONFIG
            GLOBAL_BEST_MODEL = best_model
            GLOBAL_BEST_CONFIG = best_config
            
            print(f"\n✅ Best model saved as: GLOBAL_BEST_MODEL = '{best_model}'")
    
    if unavailable_models:
        print(f"\n❌ UNAVAILABLE/FAILED MODELS ({len(unavailable_models)} models):")
        print("-" * 60)
        for model_name, result in unavailable_models.items():
            error_msg = result.get('error', 'Unknown error')[:50]
            print(f"   {model_name:<25} {error_msg}")
    
    print("="*90)
    return best_model if available_models else None

# Analyze results and select best model
selected_best_model = analyze_comprehensive_results(comprehensive_results)

if selected_best_model:
    print(f"\n🎉 READY FOR DETAILED TESTING WITH: {GLOBAL_BEST_CONFIG['name']}")
    print(f"   Next step: Run detailed test with {BEST_MODEL_TEST_SIZE} questions")
else:
    print(f"\n⚠️  No models available for testing. Check your API access.")
    # Fallback to a known working model
    GLOBAL_BEST_MODEL = "gpt-4o-mini"
    GLOBAL_BEST_CONFIG = get_model_config(GLOBAL_BEST_MODEL)
    print(f"   Falling back to: {GLOBAL_BEST_MODEL}")

In [ ]:
# ====================================================================
# DETAILED TESTING WITH BEST MODEL
# ====================================================================

async def run_detailed_test_with_best_model():
    """Run detailed test with the best performing model"""
    
    if 'GLOBAL_BEST_MODEL' not in globals():
        print("❌ No best model selected. Please run the comprehensive test first.")
        return None
    
    print(f"🚀 DETAILED TESTING WITH BEST MODEL")
    print("="*70)
    print(f"📋 Model: {GLOBAL_BEST_CONFIG['name']} ({GLOBAL_BEST_MODEL})")
    print(f"📊 Sample size: {BEST_MODEL_TEST_SIZE} questions")
    print(f"🎯 Using enhanced forced-selection prompt")
    print(f"💰 Pricing: ${GLOBAL_BEST_CONFIG['pricing']['input']:.2f}/${GLOBAL_BEST_CONFIG['pricing']['output']:.2f} per 1M tokens")
    print("="*70)
    
    # Get test sample
    test_data = df_questions_topics.head(BEST_MODEL_TEST_SIZE).to_dict('records')
    
    # Run classification
    start_time = time.time()
    sem = asyncio.Semaphore(OPTIMAL_CONCURRENCY)
    
    async def detailed_worker(question_data):
        async with sem:
            question = question_data['question']
            correct_topic = question_data['cluster_topic']
            
            predicted_topic = await classify_with_forced_selection(question, topics_list, GLOBAL_BEST_CONFIG)
            
            return {
                'question': question,
                'correct_topic': correct_topic,
                'predicted_topic': predicted_topic,
                'is_correct': predicted_topic == correct_topic
            }
    
    print(f"🔄 Processing {len(test_data)} questions...")
    tasks = [detailed_worker(q_data) for q_data in test_data]
    detailed_results = await asyncio.gather(*tasks)
    end_time = time.time()
    
    # Calculate comprehensive metrics
    total_questions = len(detailed_results)
    correct_count = sum(1 for r in detailed_results if r['is_correct'])
    accuracy = (correct_count / total_questions) * 100
    processing_time = end_time - start_time
    
    # Estimate actual cost
    avg_input_tokens = 450  # More accurate estimate for detailed prompt
    avg_output_tokens = 25  # Response tokens
    total_input = avg_input_tokens * total_questions
    total_output = avg_output_tokens * total_questions
    actual_cost = estimate_cost(total_input, total_output, GLOBAL_BEST_MODEL)
    
    print(f"\n" + "="*70)
    print(f"🏆 DETAILED TEST RESULTS - {GLOBAL_BEST_CONFIG['name']}")
    print("="*70)
    print(f"📊 Total questions: {total_questions}")
    print(f"✅ Correct predictions: {correct_count}")
    print(f"❌ Incorrect predictions: {total_questions - correct_count}")
    print(f"🎯 Accuracy: {accuracy:.1f}%")
    print(f"⚡ Processing time: {processing_time:.1f} seconds")
    print(f"📈 Questions per second: {total_questions / processing_time:.1f}")
    print(f"💸 Actual cost: ${actual_cost:.4f}")
    
    # Performance assessment
    if accuracy >= 90:
        print(f"\n🎉 EXCELLENT! {GLOBAL_BEST_CONFIG['name']} achieved {accuracy:.1f}% accuracy (≥90%)")
        print("✅ RECOMMENDATION: Use OpenAI classification (skip clustering)")
        recommendation = "USE_OPENAI"
    elif accuracy >= 75:
        print(f"\n🌟 VERY GOOD! {GLOBAL_BEST_CONFIG['name']} achieved {accuracy:.1f}% accuracy")
        print("⚡ RECOMMENDATION: Consider hybrid approach or proceed with OpenAI")
        recommendation = "HYBRID_OR_OPENAI"
    elif accuracy >= 60:
        print(f"\n📈 GOOD PROGRESS! {GLOBAL_BEST_CONFIG['name']} achieved {accuracy:.1f}% accuracy")
        print("🔧 RECOMMENDATION: Optimize prompts or use hybrid approach")
        recommendation = "OPTIMIZE_OR_HYBRID"
    else:
        print(f"\n🔄 NEEDS IMPROVEMENT: {GLOBAL_BEST_CONFIG['name']} achieved {accuracy:.1f}% accuracy")
        print("🛠️  RECOMMENDATION: Stick with clustering approach")
        recommendation = "USE_CLUSTERING"
    
    # Cost projections for full dataset
    if RUN_FULL_DATASET_TEST:
        full_cost = estimate_cost(
            avg_input_tokens * len(df_questions_topics),
            avg_output_tokens * len(df_questions_topics),
            GLOBAL_BEST_MODEL
        )
        print(f"\n💰 FULL DATASET PROJECTION:")
        print(f"   📊 Total questions: {len(df_questions_topics)}")
        print(f"   💸 Estimated cost: ${full_cost:.2f}")
        print(f"   ⏱️ Estimated time: {(len(df_questions_topics) / (total_questions / processing_time)):.0f} seconds")
    
    # Error analysis (show a few misclassifications)
    incorrect_results = [r for r in detailed_results if not r['is_correct']]
    if incorrect_results:
        print(f"\n🔍 SAMPLE MISCLASSIFICATIONS (showing 5 of {len(incorrect_results)}):")
        print("-" * 70)
        for i, result in enumerate(incorrect_results[:5], 1):
            print(f"{i}. Question: {result['question'][:60]}...")
            print(f"   Expected: {result['correct_topic']}")
            print(f"   Predicted: {result['predicted_topic']}")
            print()
    
    print("="*70)
    
    # Store results for potential full dataset test
    global DETAILED_TEST_RESULTS
    DETAILED_TEST_RESULTS = {
        'accuracy': accuracy,
        'recommendation': recommendation,
        'cost_per_question': actual_cost / total_questions,
        'processing_time': processing_time
    }
    
    return detailed_results

# Run detailed test
detailed_test_results = await run_detailed_test_with_best_model()

In [ ]:
# ====================================================================
# FULL DATASET TESTING (Conditional)
# ====================================================================

async def run_full_dataset_test():
    """Run test on all questions if enabled"""
    
    if not RUN_FULL_DATASET_TEST:
        print("ℹ️  FULL DATASET TEST DISABLED")
        print("="*50)
        print("To enable full dataset testing:")
        print("1. Set RUN_FULL_DATASET_TEST = True in the configuration")
        print("2. Re-run this cell")
        print(f"\nCurrent settings:")
        print(f"   🔧 RUN_FULL_DATASET_TEST = {RUN_FULL_DATASET_TEST}")
        print(f"   📊 Total questions available: {len(df_questions_topics)}")
        
        if 'DETAILED_TEST_RESULTS' in globals():
            projected_cost = DETAILED_TEST_RESULTS['cost_per_question'] * len(df_questions_topics)
            projected_time = (len(df_questions_topics) / 100) * DETAILED_TEST_RESULTS['processing_time']
            print(f"   💸 Projected cost: ${projected_cost:.2f}")
            print(f"   ⏱️ Projected time: {projected_time:.0f} seconds ({projected_time/60:.1f} minutes)")
        
        print("="*50)
        return None
    
    if 'GLOBAL_BEST_MODEL' not in globals():
        print("❌ No best model selected. Please run the comprehensive test first.")
        return None
    
    print(f"🚀 FULL DATASET TESTING")
    print("="*80)
    print(f"📋 Model: {GLOBAL_BEST_CONFIG['name']} ({GLOBAL_BEST_MODEL})")
    print(f"📊 Total questions: {len(df_questions_topics)}")
    print(f"🎯 Using optimized forced-selection approach")
    print(f"⚡ Concurrency: {OPTIMAL_CONCURRENCY} parallel requests")
    
    # Cost and time estimates
    if 'DETAILED_TEST_RESULTS' in globals():
        projected_cost = DETAILED_TEST_RESULTS['cost_per_question'] * len(df_questions_topics)
        projected_time = (len(df_questions_topics) / 100) * DETAILED_TEST_RESULTS['processing_time']
        print(f"💸 Estimated cost: ${projected_cost:.2f}")
        print(f"⏱️ Estimated time: {projected_time:.0f} seconds ({projected_time/60:.1f} minutes)")
    
    print("="*80)
    
    # Confirm with user (in a real scenario)
    print("⚠️  WARNING: This will test ALL questions and incur API costs!")
    print("🔄 Starting full dataset test...")
    
    # Get all question data
    all_question_data = df_questions_topics.to_dict('records')
    
    # Run classification with progress tracking
    start_time = time.time()
    sem = asyncio.Semaphore(OPTIMAL_CONCURRENCY)
    
    # Progress tracking
    completed = 0
    total = len(all_question_data)
    
    async def full_dataset_worker(question_data):
        nonlocal completed
        async with sem:
            question = question_data['question']
            correct_topic = question_data['cluster_topic']
            
            predicted_topic = await classify_with_forced_selection(question, topics_list, GLOBAL_BEST_CONFIG)
            
            completed += 1
            if completed % 100 == 0:  # Progress every 100 questions
                elapsed = time.time() - start_time
                rate = completed / elapsed
                remaining = (total - completed) / rate
                print(f"   📈 Progress: {completed}/{total} ({100*completed/total:.1f}%) - ETA: {remaining:.0f}s")
            
            return {
                'question': question,
                'correct_topic': correct_topic,
                'predicted_topic': predicted_topic,
                'is_correct': predicted_topic == correct_topic
            }
    
    print(f"🔄 Processing {total} questions...")
    tasks = [full_dataset_worker(q_data) for q_data in all_question_data]
    full_results = await asyncio.gather(*tasks)
    end_time = time.time()
    
    # Calculate final metrics
    total_questions = len(full_results)
    correct_count = sum(1 for r in full_results if r['is_correct'])
    final_accuracy = (correct_count / total_questions) * 100
    total_processing_time = end_time - start_time
    
    # Calculate actual cost
    avg_input_tokens = 450
    avg_output_tokens = 25
    total_input = avg_input_tokens * total_questions
    total_output = avg_output_tokens * total_questions
    total_cost = estimate_cost(total_input, total_output, GLOBAL_BEST_MODEL)
    
    print(f"\n" + "="*80)
    print(f"🏆 FINAL RESULTS - FULL DATASET")
    print("="*80)
    print(f"📊 Total questions processed: {total_questions}")
    print(f"✅ Correct predictions: {correct_count}")
    print(f"❌ Incorrect predictions: {total_questions - correct_count}")
    print(f"🎯 Final accuracy: {final_accuracy:.1f}%")
    print(f"⚡ Total processing time: {total_processing_time:.1f} seconds ({total_processing_time/60:.1f} minutes)")
    print(f"📈 Questions per second: {total_questions / total_processing_time:.1f}")
    print(f"💸 Total cost: ${total_cost:.2f}")
    
    # Final recommendation
    print(f"\n🏆 FINAL DECISION:")
    if final_accuracy >= 90:
        print(f"🎉 EXCELLENT! {final_accuracy:.1f}% accuracy achieved!")
        print("✅ STRONG RECOMMENDATION: Replace clustering with OpenAI classification")
        print("💡 Benefits: Faster processing, no training needed, consistent results")
    elif final_accuracy >= 75:
        print(f"🌟 VERY GOOD! {final_accuracy:.1f}% accuracy achieved!")
        print("⚡ RECOMMENDATION: Consider OpenAI classification or hybrid approach")
        print("💡 OpenAI classification is viable for this use case")
    elif final_accuracy >= 60:
        print(f"📈 MODERATE SUCCESS: {final_accuracy:.1f}% accuracy achieved")
        print("🔧 RECOMMENDATION: Optimize prompts or use hybrid approach")
        print("💡 Clustering + OpenAI fallback might be optimal")
    else:
        print(f"🔄 BELOW TARGET: {final_accuracy:.1f}% accuracy achieved")
        print("🛠️  RECOMMENDATION: Continue with clustering approach")
        print("💡 OpenAI not yet suitable for replacing clustering")
    
    # Comparison with clustering baseline
    print(f"\n📊 PERFORMANCE COMPARISON:")
    print(f"   🤖 OpenAI ({GLOBAL_BEST_CONFIG['name']}): {final_accuracy:.1f}%")
    print(f"   🧩 Clustering baseline: ~85-95% (estimated)")
    print(f"   📈 Gap: {final_accuracy - 90:.1f} percentage points from target")
    
    print("="*80)
    
    return full_results

# Run full dataset test (conditional)
full_dataset_results = await run_full_dataset_test()

## Comprehensive Testing with Enhanced Prompt

This section implements the most advanced classification approach with enhanced prompts, keyword fallback strategies, and optimal settings to achieve the highest possible accuracy.

In [ ]:
async def classify_question_comprehensive(question: str, available_topics: List[str]) -> str:
    """
    Comprehensive classification approach with enhanced prompt and fallback strategies
    """
    topics_str = "\n".join([f"• {topic}" for topic in available_topics])
    
    prompt = f"""You are an expert at classifying BYU-Pathway student questions into specific topics.

AVAILABLE TOPICS:
{topics_str}
• Other (only if nothing else fits)

CLASSIFICATION RULES:
1. Look for keywords and context in the question
2. Choose the MOST SPECIFIC topic that matches
3. Key patterns:
   - "religion courses" → Religion Course Registration and Requirements
   - "certificate" or "completion" → Certificate Application Process  
   - "scholarship" or "Heber" → Heber J. Grant Scholarship Application Process
   - "BYU Pathway" or "accredited" → BYU-Pathway Program
   - "English Connect 3" or "EC3" → English Connect 3 Registration Process
   - "registration" → match to most specific registration topic
   - "portal" or "login" → appropriate Student Portal topic
4. Avoid "Other" unless truly nothing fits

QUESTION: "{question}"

Respond with ONLY the exact topic name from the list above:"""

    try:
        response = await async_client.chat.completions.create(
            model=OPTIMAL_MODEL,
            messages=[
                {"role": "system", "content": "You are an expert classifier. Choose the most specific matching topic. Avoid 'Other' when possible."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0
        )
        
        result = response.choices[0].message.content.strip()
        
        # Clean up response and find exact match
        result_clean = result.replace("•", "").replace("-", "").strip().rstrip('.')
        
        # Direct match
        if result_clean in available_topics:
            return result_clean
        
        # Fuzzy matching for partial responses
        for topic in available_topics:
            if topic.lower() in result_clean.lower() or result_clean.lower() in topic.lower():
                if len(result_clean) > 5:  # Avoid very short matches
                    return topic
        
        # Keyword fallback strategies
        question_lower = question.lower()
        if "religion" in question_lower and "course" in question_lower:
            return "Religion Course Registration and Requirements"
        elif "scholarship" in question_lower or "heber" in question_lower:
            return "Heber J. Grant Scholarship Application Process"
        elif ("byu pathway" in question_lower or "accredited" in question_lower):
            return "BYU-Pathway Program"
        elif ("english connect 3" in question_lower or "ec3" in question_lower):
            return "English Connect 3 Registration Process"
        elif "certificate" in question_lower and ("completion" in question_lower or "complete" in question_lower):
            return "Certificate Application Process"
        
        return "Other"
        
    except Exception as e:
        print(f"⚠️  Error classifying question: {e}")
        return "Other"

async def worker_comprehensive(question_data: Dict, sem: asyncio.Semaphore, available_topics: List[str]) -> Dict[str, Any]:
    """
    Comprehensive worker with optimal settings and error handling
    """
    async with sem:
        question = question_data['question']
        correct_topic = question_data['cluster_topic']
        
        openai_topic = await classify_question_comprehensive(question, available_topics)
        
        return {
            'question': question,
            'correct_topic': correct_topic,
            'openai_topic': openai_topic,
            'is_correct': openai_topic == correct_topic
        }

# Comprehensive Test on 100 Questions
print("\n🚀 COMPREHENSIVE PARALLEL TESTING")
print("="*50)
print("Testing with optimal settings and enhanced prompt on 100 questions...")

test_sample = df_questions_topics.head(100).to_dict('records')

start_time = time.time()
sem = asyncio.Semaphore(OPTIMAL_CONCURRENCY)
tasks = [asyncio.create_task(worker_comprehensive(q_data, sem, topics_list)) for q_data in test_sample]
comprehensive_results = await asyncio.gather(*tasks)
end_time = time.time()

# Calculate comprehensive results
comprehensive_correct = sum(1 for result in comprehensive_results if result['is_correct'])
comprehensive_accuracy = (comprehensive_correct / len(comprehensive_results)) * 100

print(f"\n📊 COMPREHENSIVE TEST RESULTS:")
print(f"✅ Accuracy: {comprehensive_accuracy:.1f}%")
print(f"⚡ Processing time: {end_time - start_time:.1f} seconds")
print(f"📈 Questions tested: {len(comprehensive_results)}")
print(f"✓ Correct: {comprehensive_correct}")
print(f"✗ Incorrect: {len(comprehensive_results) - comprehensive_correct}")

# Determine next steps
if comprehensive_accuracy >= 90:
    print(f"\n🏆 EXCELLENT! Achieved {comprehensive_accuracy:.1f}% accuracy (≥90%)")
    print("✅ RECOMMENDATION: Proceed with OpenAI classification (skip clustering)")
elif comprehensive_accuracy >= 80:
    print(f"\n🌟 VERY GOOD! {comprehensive_accuracy:.1f}% accuracy (close to target)")
    print("⚡ RECOMMENDATION: Consider hybrid approach or minor refinements")
elif comprehensive_accuracy >= 70:
    print(f"\n📈 GOOD PROGRESS! {comprehensive_accuracy:.1f}% accuracy")
    print("🔧 RECOMMENDATION: Continue refining prompt or use hybrid approach")
else:
    print(f"\n🔄 NEEDS MORE WORK: {comprehensive_accuracy:.1f}% accuracy")
    print("🛠️  RECOMMENDATION: Stick with clustering approach for now")

print("="*50)

In [ ]:
# Test Different Models (Including Checking for Newer Models)
print("🔍 TESTING DIFFERENT OPENAI MODELS")
print("="*50)

# List of models to test (based on confirmed availability)
models_to_test = [
    "gpt-4o-mini",           # Current best performer
    "gpt-4o",                # Previous test model
    "gpt-5",                 # ✅ CONFIRMED: Released August 7, 2025
    "gpt-5-thinking",        # GPT-5 with reasoning mode
    "gpt-5-mini",            # Mini version mentioned in GPT-5 docs
    "gpt-5-nano",            # ✅ CONFIRMED: Fastest, cheapest GPT-5 for classification
    "gpt-4-turbo",           # Alternative model
    "gpt-3.5-turbo",         # Faster/cheaper option
    "gpt-4o-2024-08-06",     # Specific version
]

print("🔍 CONFIRMED MODEL INFORMATION:")
print("✅ GPT-5: Released August 7, 2025 - 'Our smartest, fastest, most useful model yet'")
print("✅ GPT-5 Thinking: Extended reasoning version")
print("✅ GPT-5 Mini: Smaller, faster version mentioned in docs")
print("✅ GPT-5 Nano: CONFIRMED - 'Fastest, cheapest GPT-5 for classification tasks'")
print("   💰 Pricing: $0.05 input / $0.40 output per 1M tokens")
print("   🎯 Perfect for classification tasks like ours!")
print("="*50)

# Function to test if a model exists and works
async def test_model_availability(model_name: str) -> bool:
    """Test if a model is available and working"""
    try:
        response = await async_client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": "Test"}],
            max_tokens=5
        )
        return True
    except Exception as e:
        print(f"❌ {model_name}: {str(e)[:100]}...")
        return False

# Test model availability
print("Testing model availability...")
available_models = []

for model in models_to_test:
    print(f"Testing {model}...", end=" ")
    is_available = await test_model_availability(model)
    if is_available:
        print("✅ Available")
        available_models.append(model)
    else:
        print("❌ Not available")

print(f"\n✅ Available models: {available_models}")

# Test classification with available models on sample questions
if available_models:
    print(f"\n🧪 TESTING CLASSIFICATION ACCURACY WITH AVAILABLE MODELS")
    print("="*60)
    
    # Use a smaller sample for testing multiple models
    test_sample_small = df_questions_topics.head(20).to_dict('records')
    model_results = {}
    
    for model in available_models:
        print(f"\n🔬 Testing {model}...")
        
        # Modified classification function for different models
        async def classify_with_specific_model(question: str, available_topics: List[str], model: str) -> str:
            topics_str = "\n".join([f"• {topic}" for topic in available_topics])
            
            prompt = f"""You are an expert at classifying BYU-Pathway student questions into specific topics.

AVAILABLE TOPICS:
{topics_str}
• Other (only if nothing else fits)

CLASSIFICATION RULES:
1. Look for keywords and context in the question
2. Choose the MOST SPECIFIC topic that matches
3. Key patterns:
   - "religion courses" → Religion Course Registration and Requirements
   - "certificate" or "completion" → Certificate Application Process  
   - "scholarship" or "Heber" → Heber J. Grant Scholarship Application Process
   - "BYU Pathway" or "accredited" → BYU-Pathway Program
   - "English Connect 3" or "EC3" → English Connect 3 Registration Process
4. Avoid "Other" unless truly nothing fits

QUESTION: "{question}"

Respond with ONLY the exact topic name from the list above:"""

            try:
                response = await async_client.chat.completions.create(
                    model=model,
                    messages=[
                        {"role": "system", "content": "You are an expert classifier. Choose the most specific matching topic. Avoid 'Other' when possible."},
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens=100,
                    temperature=0
                )
                
                result = response.choices[0].message.content.strip()
                result_clean = result.replace("•", "").replace("-", "").strip().rstrip('.')
                
                # Find exact match
                if result_clean in available_topics:
                    return result_clean
                
                # Fuzzy matching
                for topic in available_topics:
                    if topic.lower() in result_clean.lower() or result_clean.lower() in topic.lower():
                        if len(result_clean) > 5:
                            return topic
                
                return "Other"
                
            except Exception as e:
                print(f"⚠️  Error with {model}: {e}")
                return "Other"

        # Test this model
        start_time = time.time()
        sem = asyncio.Semaphore(8)  # Lower concurrency for testing
        
        tasks = []
        for q_data in test_sample_small:
            async def worker_for_model(question_data, model_name):
                async with sem:
                    question = question_data['question']
                    correct_topic = question_data['cluster_topic']
                    
                    openai_topic = await classify_with_specific_model(question, topics_list, model_name)
                    
                    return {
                        'question': question,
                        'correct_topic': correct_topic,
                        'openai_topic': openai_topic,
                        'is_correct': openai_topic == correct_topic
                    }
            
            tasks.append(asyncio.create_task(worker_for_model(q_data, model)))
        
        try:
            results = await asyncio.gather(*tasks)
            end_time = time.time()
            
            # Calculate results
            correct = sum(1 for r in results if r['is_correct'])
            accuracy = (correct / len(results)) * 100
            processing_time = end_time - start_time
            
            model_results[model] = {
                'accuracy': accuracy,
                'correct': correct,
                'total': len(results),
                'time': processing_time
            }
            
            print(f"   ✅ Accuracy: {accuracy:.1f}% ({correct}/{len(results)})")
            print(f"   ⚡ Time: {processing_time:.1f}s")
            
        except Exception as e:
            print(f"   ❌ Failed to test {model}: {e}")
            model_results[model] = {'accuracy': 0, 'error': str(e)}

    # Summary of model comparison
    print(f"\n📊 MODEL COMPARISON SUMMARY")
    print("="*50)
    
    best_model = None
    best_accuracy = 0
    
    for model, results in model_results.items():
        if 'accuracy' in results and 'error' not in results:
            accuracy = results['accuracy']
            time_taken = results['time']
            print(f"{model:20} | {accuracy:5.1f}% | {time_taken:4.1f}s")
            
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model
        else:
            print(f"{model:20} | ERROR")
    
    if best_model:
        print(f"\n🏆 BEST PERFORMING MODEL: {best_model} ({best_accuracy:.1f}% accuracy)")
        print(f"💡 Recommendation: Use {best_model} for full testing")
    
else:
    print("❌ No models available for testing!")

In [ ]:
# Full Test with Best Model on All Questions
if 'best_model' in locals() and best_model:
    print(f"\n🚀 FULL TEST WITH BEST MODEL: {best_model}")
    print("="*60)
    print(f"Testing on ALL {len(df_questions_topics)} questions...")
    
    # Use the best model for comprehensive testing
    BEST_MODEL = best_model
    
    async def classify_with_best_model(question: str, available_topics: List[str]) -> str:
        """Classification using the best performing model"""
        topics_str = "\n".join([f"• {topic}" for topic in available_topics])
        
        prompt = f"""You are an expert at classifying BYU-Pathway student questions into specific topics.

AVAILABLE TOPICS:
{topics_str}
• Other (only if nothing else fits)

CLASSIFICATION RULES:
1. Look for keywords and context in the question
2. Choose the MOST SPECIFIC topic that matches
3. Key patterns:
   - "religion courses" → Religion Course Registration and Requirements
   - "certificate" or "completion" → Certificate Application Process  
   - "scholarship" or "Heber" → Heber J. Grant Scholarship Application Process
   - "BYU Pathway" or "accredited" → BYU-Pathway Program
   - "English Connect 3" or "EC3" → English Connect 3 Registration Process
   - "registration" → match to most specific registration topic
   - "portal" or "login" → appropriate Student Portal topic
4. Avoid "Other" unless truly nothing fits

QUESTION: "{question}"

Respond with ONLY the exact topic name from the list above:"""

        try:
            # Use different parameters based on model type
            if BEST_MODEL.startswith('gpt-5'):
                # GPT-5 models use max_completion_tokens and don't support temperature=0
                response = await async_client.chat.completions.create(
                    model=BEST_MODEL,
                    messages=[
                        {"role": "system", "content": "You are an expert classifier. Choose the most specific matching topic. Avoid 'Other' when possible."},
                        {"role": "user", "content": prompt}
                    ],
                    max_completion_tokens=100
                )
            else:
                # GPT-4 models use max_tokens and support temperature=0
                response = await async_client.chat.completions.create(
                    model=BEST_MODEL,
                    messages=[
                        {"role": "system", "content": "You are an expert classifier. Choose the most specific matching topic. Avoid 'Other' when possible."},
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens=100,
                    temperature=0
                )
            
            result = response.choices[0].message.content.strip()
            result_clean = result.replace("•", "").replace("-", "").strip().rstrip('.')
            
            # Direct match
            if result_clean in available_topics:
                return result_clean
            
            # Fuzzy matching for partial responses
            for topic in available_topics:
                if topic.lower() in result_clean.lower() or result_clean.lower() in topic.lower():
                    if len(result_clean) > 5:
                        return topic
            
            # Keyword fallback strategies
            question_lower = question.lower()
            if "religion" in question_lower and "course" in question_lower:
                return "Religion Course Registration and Requirements"
            elif "scholarship" in question_lower or "heber" in question_lower:
                return "Heber J. Grant Scholarship Application Process"
            elif ("byu pathway" in question_lower or "accredited" in question_lower):
                return "BYU-Pathway Program"
            elif ("english connect 3" in question_lower or "ec3" in question_lower):
                return "English Connect 3 Registration Process"
            elif "certificate" in question_lower and ("completion" in question_lower or "complete" in question_lower):
                return "Certificate Application Process"
            
            return "Other"
            
        except Exception as e:
            print(f"⚠️  Error classifying question: {e}")
            return "Other"

    async def worker_best_model(question_data: Dict, sem: asyncio.Semaphore, available_topics: List[str]) -> Dict[str, Any]:
        """Worker using the best model"""
        async with sem:
            question = question_data['question']
            correct_topic = question_data['cluster_topic']
            
            openai_topic = await classify_with_best_model(question, available_topics)
            
            return {
                'question': question,
                'correct_topic': correct_topic,
                'openai_topic': openai_topic,
                'is_correct': openai_topic == correct_topic
            }

    # Run full test with best model
    all_questions_data = df_questions_topics.to_dict('records')
    
    start_time = time.time()
    sem = asyncio.Semaphore(OPTIMAL_CONCURRENCY)  # Use optimal concurrency
    tasks = [asyncio.create_task(worker_best_model(q_data, sem, topics_list)) for q_data in all_questions_data]
    
    print(f"🔄 Processing {len(all_questions_data)} questions with {BEST_MODEL}...")
    best_model_results = await asyncio.gather(*tasks)
    end_time = time.time()

    # Calculate final results
    total_questions = len(best_model_results)
    correct_count = sum(1 for result in best_model_results if result['is_correct'])
    final_accuracy = (correct_count / total_questions) * 100
    processing_time = end_time - start_time

    print(f"\n" + "="*60)
    print(f"🏆 FINAL RESULTS WITH {BEST_MODEL}")
    print("="*60)
    print(f"📊 Total questions: {total_questions}")
    print(f"✅ Correct: {correct_count}")
    print(f"❌ Incorrect: {total_questions - correct_count}")
    print(f"🎯 Accuracy: {final_accuracy:.1f}%")
    print(f"⚡ Processing time: {processing_time:.1f} seconds")
    print(f"📈 Questions per second: {total_questions / processing_time:.1f}")

    # Decision based on results
    if final_accuracy >= 90:
        print(f"\n🎉 EXCELLENT! {BEST_MODEL} achieved {final_accuracy:.1f}% accuracy (≥90%)")
        print("✅ RECOMMENDATION: Use OpenAI classification with this model (skip clustering)")
    elif final_accuracy >= 80:
        print(f"\n🌟 VERY GOOD! {BEST_MODEL} achieved {final_accuracy:.1f}% accuracy")
        print("⚡ RECOMMENDATION: Consider hybrid approach or minor refinements")
    elif final_accuracy >= 70:
        print(f"\n📈 GOOD PROGRESS! {BEST_MODEL} achieved {final_accuracy:.1f}% accuracy")
        print("🔧 RECOMMENDATION: Continue refining or use hybrid approach")
    else:
        print(f"\n🔄 NEEDS MORE WORK: {BEST_MODEL} achieved {final_accuracy:.1f}% accuracy")
        print("🛠️  RECOMMENDATION: Stick with clustering approach")

    # Compare with previous best result
    print(f"\n📈 COMPARISON:")
    print(f"   Previous best (gpt-4o-mini): 53.2%")
    print(f"   {BEST_MODEL}: {final_accuracy:.1f}%")
    improvement = final_accuracy - 53.2
    print(f"   Improvement: {improvement:+.1f} percentage points")

    print("="*60)

else:
    print("❌ No best model identified. Please run the model testing cell first.")

In [ ]:
# GPT-5 Nano Optimized Test
if 'best_model' in locals() and best_model and best_model.startswith('gpt-5'):
    print(f"\n🚀 GPT-5 NANO OPTIMIZED TEST")
    print("="*60)
    print(f"Running optimized test with {best_model} using correct GPT-5 parameters...")
    print(f"📊 Testing on {len(df_questions_topics)} questions")
    print(f"⚙️  Using max_completion_tokens (not max_tokens) and no temperature parameter")
    
    async def classify_with_gpt5_nano(question: str, available_topics: List[str]) -> str:
        """GPT-5 nano optimized classification function"""
        topics_str = "\n".join([f"• {topic}" for topic in available_topics])
        
        prompt = f"""You are an expert at classifying BYU-Pathway student questions into specific topics.

AVAILABLE TOPICS:
{topics_str}
• Other (only if nothing else fits)

CLASSIFICATION RULES:
1. Look for keywords and context in the question
2. Choose the MOST SPECIFIC topic that matches
3. Key patterns:
   - "religion courses" → Religion Course Registration and Requirements
   - "certificate" or "completion" → Certificate Application Process  
   - "scholarship" or "Heber" → Heber J. Grant Scholarship Application Process
   - "BYU Pathway" or "accredited" → BYU-Pathway Program
   - "English Connect 3" or "EC3" → English Connect 3 Registration Process
4. Avoid "Other" unless truly nothing fits

QUESTION: "{question}"

Respond with ONLY the exact topic name from the list above:"""

        try:
            # GPT-5 nano specific parameters
            response = await async_client.chat.completions.create(
                model=best_model,
                messages=[
                    {"role": "system", "content": "You are an expert classifier. Respond with only the exact topic name."},
                    {"role": "user", "content": prompt}
                ],
                max_completion_tokens=200  # Increased for longer responses
                # No temperature parameter as it's not supported
            )
            
            result = response.choices[0].message.content.strip()
            result_clean = result.replace("•", "").replace("-", "").strip().rstrip('.')
            
            # Direct match
            if result_clean in available_topics:
                return result_clean
            
            # Fuzzy matching for partial responses
            for topic in available_topics:
                if topic.lower() in result_clean.lower() or result_clean.lower() in topic.lower():
                    if len(result_clean) > 5:
                        return topic
            
            # Keyword fallback strategies
            question_lower = question.lower()
            if "religion" in question_lower and "course" in question_lower:
                return "Religion Course Registration and Requirements"
            elif "scholarship" in question_lower or "heber" in question_lower:
                return "Heber J. Grant Scholarship Application Process"
            elif ("byu pathway" in question_lower or "accredited" in question_lower):
                return "BYU-Pathway Program"
            elif ("english connect 3" in question_lower or "ec3" in question_lower):
                return "English Connect 3 Registration Process"
            elif "certificate" in question_lower and ("completion" in question_lower or "complete" in question_lower):
                return "Certificate Application Process"
            
            return "Other"
            
        except Exception as e:
            print(f"⚠️  Error classifying question: {e}")
            return "Other"

    async def worker_gpt5_nano(question_data: Dict, sem: asyncio.Semaphore, available_topics: List[str]) -> Dict[str, Any]:
        """Worker optimized for GPT-5 nano"""
        async with sem:
            question = question_data['question']
            correct_topic = question_data['cluster_topic']
            
            openai_topic = await classify_with_gpt5_nano(question, available_topics)
            
            return {
                'question': question,
                'correct_topic': correct_topic,
                'openai_topic': openai_topic,
                'is_correct': openai_topic == correct_topic
            }

    # Run optimized test
    all_questions_data = df_questions_topics.to_dict('records')
    
    start_time = time.time()
    sem = asyncio.Semaphore(OPTIMAL_CONCURRENCY)  # Use optimal concurrency
    tasks = [asyncio.create_task(worker_gpt5_nano(q_data, sem, topics_list)) for q_data in all_questions_data]
    
    print(f"🔄 Processing {len(all_questions_data)} questions with {best_model}...")
    gpt5_nano_results = await asyncio.gather(*tasks)
    end_time = time.time()

    # Calculate final results
    total_questions = len(gpt5_nano_results)
    correct_count = sum(1 for result in gpt5_nano_results if result['is_correct'])
    gpt5_accuracy = (correct_count / total_questions) * 100
    processing_time = end_time - start_time

    print(f"\n" + "="*60)
    print(f"🏆 GPT-5 NANO RESULTS")
    print("="*60)
    print(f"📊 Total questions: {total_questions}")
    print(f"✅ Correct: {correct_count}")
    print(f"❌ Incorrect: {total_questions - correct_count}")
    print(f"🎯 Accuracy: {gpt5_accuracy:.1f}%")
    print(f"⚡ Processing time: {processing_time:.1f} seconds")
    print(f"📈 Questions per second: {total_questions / processing_time:.1f}")
    print(f"💰 Cost efficiency: {gpt5_accuracy:.1f}% accuracy at $0.05/$0.40 per 1M tokens")

    # Decision based on results
    if gpt5_accuracy >= 90:
        print(f"\n🎉 EXCELLENT! GPT-5 nano achieved {gpt5_accuracy:.1f}% accuracy (≥90%)")
        print("✅ RECOMMENDATION: Use GPT-5 nano for production (skip clustering)")
        print("💡 Benefits: Highest accuracy + lowest cost + fastest speed")
    elif gpt5_accuracy >= 80:
        print(f"\n🌟 VERY GOOD! GPT-5 nano achieved {gpt5_accuracy:.1f}% accuracy")
        print("⚡ RECOMMENDATION: Consider GPT-5 nano with minor refinements")
    elif gpt5_accuracy >= 70:
        print(f"\n📈 GOOD PROGRESS! GPT-5 nano achieved {gpt5_accuracy:.1f}% accuracy")
        print("🔧 RECOMMENDATION: Continue refining or use hybrid approach")
    else:
        print(f"\n🔄 NEEDS MORE WORK: GPT-5 nano achieved {gpt5_accuracy:.1f}% accuracy")
        print("🛠️  RECOMMENDATION: Consider other approaches")

    # Compare with previous results
    print(f"\n📈 COMPREHENSIVE COMPARISON:")
    print(f"   GPT-4o-mini baseline: 53.2%")
    print(f"   GPT-5 nano (optimized): {gpt5_accuracy:.1f}%")
    improvement = gpt5_accuracy - 53.2
    print(f"   Improvement: {improvement:+.1f} percentage points")
    
    if improvement > 0:
        print(f"\n💡 GPT-5 nano is {'SIGNIFICANTLY' if improvement > 20 else 'MODERATELY' if improvement > 10 else 'SLIGHTLY'} better!")
        print(f"   + Higher accuracy: {improvement:+.1f}%")
        print(f"   + Lower cost: $0.05 vs $0.10+ input cost")
        print(f"   + Faster speed: Optimized for classification")
        print(f"   + Larger context: 400K tokens vs 128K")

    print("="*60)

else:
    print("ℹ️  GPT-5 nano test skipped - model not available or not selected as best model")

In [ ]:
# GPT-5 Nano Simplified Test (Minimal Prompt for Better Performance)
if 'best_model' in locals() and best_model and best_model.startswith('gpt-5'):
    print(f"\n🚀 GPT-5 NANO SIMPLIFIED TEST")
    print("="*60)
    print(f"Testing {best_model} with simplified prompt for better accuracy...")
    print(f"📊 Testing on {len(df_questions_topics)} questions")
    print(f"🎯 Using minimal prompt optimized for classification models")
    
    async def classify_simplified_gpt5(question: str, available_topics: List[str]) -> str:
        """Simplified GPT-5 nano classification - minimal prompt for best performance"""
        
        # Create a simpler, more direct prompt
        topics_numbered = "\n".join([f"{i+1}. {topic}" for i, topic in enumerate(available_topics)])
        
        prompt = f"""Classify this student question into one of these topics:

{topics_numbered}
{len(available_topics) + 1}. Other

Question: "{question}"

Respond with only the number and topic name (e.g., "1. Course Registration"):"""

        try:
            response = await async_client.chat.completions.create(
                model=best_model,
                messages=[{"role": "user", "content": prompt}],
                max_completion_tokens=50  # Smaller since we just need topic name
            )
            
            result = response.choices[0].message.content.strip()
            
            # Extract topic from numbered response
            # Look for patterns like "1. Topic Name" or just "Topic Name"
            for i, topic in enumerate(available_topics):
                if f"{i+1}." in result or topic.lower() in result.lower():
                    return topic
            
            # Fallback - try fuzzy matching
            for topic in available_topics:
                if any(word in topic.lower() for word in result.lower().split() if len(word) > 3):
                    return topic
            
            # If "other" is mentioned
            if "other" in result.lower():
                return "Other"
            
            # Last resort keyword matching
            question_lower = question.lower()
            if "religion" in question_lower and "course" in question_lower:
                return "Religion Course Registration and Requirements"
            elif "scholarship" in question_lower or "heber" in question_lower:
                return "Heber J. Grant Scholarship Application Process"
            elif "certificate" in question_lower:
                return "Certificate Application Process"
            
            return "Other"
            
        except Exception as e:
            if "max_tokens" in str(e) or "output limit" in str(e):
                # Try even simpler approach for problematic questions
                try:
                    simple_prompt = f"Topic for: {question[:100]}?\n\nOptions: {', '.join(available_topics[:5])}..."
                    response = await async_client.chat.completions.create(
                        model=best_model,
                        messages=[{"role": "user", "content": simple_prompt}],
                        max_completion_tokens=20
                    )
                    result = response.choices[0].message.content.strip()
                    
                    # Find best match
                    for topic in available_topics:
                        if topic.lower() in result.lower():
                            return topic
                    return "Other"
                except:
                    return "Other"
            else:
                print(f"⚠️  Error: {str(e)[:100]}...")
                return "Other"

    async def worker_simplified_gpt5(question_data: Dict, sem: asyncio.Semaphore, available_topics: List[str]) -> Dict[str, Any]:
        """Simplified worker for GPT-5 nano"""
        async with sem:
            question = question_data['question']
            correct_topic = question_data['cluster_topic']
            
            openai_topic = await classify_simplified_gpt5(question, available_topics)
            
            return {
                'question': question,
                'correct_topic': correct_topic,
                'openai_topic': openai_topic,
                'is_correct': openai_topic == correct_topic
            }

    # Run simplified test
    all_questions_data = df_questions_topics.to_dict('records')
    
    start_time = time.time()
    sem = asyncio.Semaphore(OPTIMAL_CONCURRENCY)
    tasks = [asyncio.create_task(worker_simplified_gpt5(q_data, sem, topics_list)) for q_data in all_questions_data]
    
    print(f"🔄 Processing {len(all_questions_data)} questions with simplified approach...")
    simplified_results = await asyncio.gather(*tasks)
    end_time = time.time()

    # Calculate results
    total_questions = len(simplified_results)
    correct_count = sum(1 for result in simplified_results if result['is_correct'])
    simplified_accuracy = (correct_count / total_questions) * 100
    processing_time = end_time - start_time

    print(f"\n" + "="*60)
    print(f"🏆 GPT-5 NANO SIMPLIFIED RESULTS")
    print("="*60)
    print(f"📊 Total questions: {total_questions}")
    print(f"✅ Correct: {correct_count}")
    print(f"❌ Incorrect: {total_questions - correct_count}")
    print(f"🎯 Accuracy: {simplified_accuracy:.1f}%")
    print(f"⚡ Processing time: {processing_time:.1f} seconds")
    print(f"📈 Questions per second: {total_questions / processing_time:.1f}")
    print(f"💰 Estimated cost: ~${(total_questions * 50 / 1000000) * 0.05:.4f} (very low!)")

    # Analysis and recommendations
    if simplified_accuracy >= 90:
        print(f"\n🎉 OUTSTANDING! GPT-5 nano achieved {simplified_accuracy:.1f}% accuracy (≥90%)")
        print("✅ RECOMMENDATION: Use GPT-5 nano for production (skip clustering entirely)")
        print("🚀 Benefits:")
        print(f"   • Highest accuracy: {simplified_accuracy:.1f}%")
        print(f"   • Lowest cost: $0.05 input (10x cheaper than GPT-4)")
        print(f"   • Fastest speed: {total_questions / processing_time:.1f} q/sec")
        print(f"   • Largest context: 400K tokens")
    elif simplified_accuracy >= 80:
        print(f"\n🌟 EXCELLENT! GPT-5 nano achieved {simplified_accuracy:.1f}% accuracy")
        print("⚡ RECOMMENDATION: Use GPT-5 nano with confidence")
    elif simplified_accuracy >= 70:
        print(f"\n📈 VERY GOOD! GPT-5 nano achieved {simplified_accuracy:.1f}% accuracy")
        print("🔧 RECOMMENDATION: Consider refinements or hybrid approach")
    else:
        print(f"\n🔄 GOOD PROGRESS: GPT-5 nano achieved {simplified_accuracy:.1f}% accuracy")
        print("🛠️  RECOMMENDATION: Continue optimizing or consider alternatives")

    # Final comparison
    print(f"\n📈 FINAL COMPARISON CHART:")
    print(f"   GPT-4o-mini baseline:     53.2%")
    print(f"   GPT-5 nano (simplified):  {simplified_accuracy:.1f}%")
    improvement = simplified_accuracy - 53.2
    print(f"   Net improvement:          {improvement:+.1f} percentage points")
    
    if improvement > 30:
        print(f"\n🏆 BREAKTHROUGH RESULT! GPT-5 nano is dramatically better!")
    elif improvement > 20:
        print(f"\n🎯 EXCELLENT IMPROVEMENT! GPT-5 nano significantly outperforms baseline!")
    elif improvement > 10:
        print(f"\n📈 GOOD IMPROVEMENT! GPT-5 nano shows solid gains!")
    elif improvement > 0:
        print(f"\n✅ POSITIVE RESULT! GPT-5 nano performs better than baseline!")
    
    print("\n💡 FINAL DECISION:")
    if simplified_accuracy >= 85:
        print("   → IMPLEMENT GPT-5 nano classification in production")
        print("   → Skip clustering step entirely")
        print("   → Massive cost savings + higher accuracy")
    elif simplified_accuracy >= 75:
        print("   → Consider GPT-5 nano for production with monitoring")
        print("   → May still benefit from some prompt refinements")
    else:
        print("   → Continue with clustering approach")
        print("   → GPT-5 nano needs more optimization for this task")

    print("="*60)

else:
    print("ℹ️  GPT-5 nano simplified test skipped - not using GPT-5 model")

In [ ]:
# Test GPT-5 Nano Specifically (Optimized for Classification)
print("🚀 TESTING GPT-5 NANO AVAILABILITY")
print("="*50)
print("📋 GPT-5 Nano Specifications:")
print("   • Optimized for: Classification and summarization tasks")
print("   • Speed: Very fast")
print("   • Price: $0.05 input / $0.40 output per 1M tokens")
print("   • Context: 400,000 tokens")
print("   • Output: 128,000 max tokens")
print("="*50)

async def test_gpt5_nano():
    """Test if GPT-5 nano is available and working"""
    models_to_test = [
        "gpt-5-nano",
        "gpt-5-nano-2025-08-07",  # Specific snapshot version
    ]
    
    for model in models_to_test:
        print(f"\n🔍 Testing {model}...")
        
        # GPT-5 nano parameter combinations to try
        parameter_sets = [
            # Try with minimal parameters
            {"messages": [{"role": "user", "content": "Test classification"}]},
            # Try with temperature=1 (default)
            {"messages": [{"role": "user", "content": "Test"}], "temperature": 1},
            # Try without temperature
            {"messages": [{"role": "user", "content": "Test"}]},
            # Try with different temperature values
            {"messages": [{"role": "user", "content": "Test"}], "temperature": 0.1},
            {"messages": [{"role": "user", "content": "Test"}], "temperature": 0.5},
        ]
        
        for i, params in enumerate(parameter_sets):
            try:
                print(f"   🔧 Trying parameter set {i+1}...")
                response = await async_client.chat.completions.create(
                    model=model,
                    **params
                )
                result = response.choices[0].message.content.strip()
                print(f"   ✅ {model} is AVAILABLE!")
                print(f"   📝 Test response: '{result}'")
                print(f"   🎯 Working parameters: {params}")
                return model  # Return the working model
                
            except Exception as e:
                error_msg = str(e)
                if "does not exist" in error_msg:
                    print(f"   ❌ {model}: Model not found in your account")
                    break  # No point trying other parameter sets
                elif "temperature" in error_msg.lower():
                    print(f"   ⚠️  Parameter set {i+1}: Temperature issue - {error_msg[:100]}")
                    continue  # Try next parameter set
                elif "max_tokens" in error_msg.lower():
                    print(f"   ⚠️  Parameter set {i+1}: Max tokens issue - {error_msg[:100]}")
                    continue  # Try next parameter set
                else:
                    print(f"   ⚠️  Parameter set {i+1}: {error_msg[:100]}")
                    continue  # Try next parameter set
        
        print(f"   ❌ {model}: All parameter combinations failed")
    
    return None

# Test GPT-5 nano availability
available_gpt5_nano = await test_gpt5_nano()

if available_gpt5_nano:
    print(f"\n🎉 GPT-5 NANO IS AVAILABLE: {available_gpt5_nano}")
    print("🚀 Let's run the full test with GPT-5 nano!")
    
    # Override the best_model with GPT-5 nano for testing
    best_model = available_gpt5_nano
    BEST_MODEL = available_gpt5_nano
    
    print(f"\n🏆 UPDATED BEST MODEL: {BEST_MODEL}")
    print("   This model is specifically optimized for classification tasks!")
    
else:
    print(f"\n💡 GPT-5 nano not yet available in your account")
    print(f"   Your account may not have access to GPT-5 models yet")
    print(f"   Continuing with current best model: gpt-4o-mini")
    
    # Keep the existing best model
    if 'best_model' not in locals():
        best_model = "gpt-4o-mini"  # Fallback to known working model
        BEST_MODEL = best_model

: 

## Summary and Conclusions

### What We Tested:
1. **Current Process**: Questions → Embeddings → Clustering → Natural Language Topics
2. **Proposed Process**: Questions + Topic List → Direct OpenAI Classification (skip clustering)

### Key Findings:
- **Optimal Model**: GPT-4o-mini (better reliability than GPT-4o)
- **Optimal Concurrency**: 16 parallel requests (balance of speed and API stability)
- **Best Approach**: Comprehensive prompt with keyword fallbacks

### Decision Framework:
- **If accuracy ≥ 90%**: Skip clustering, use direct OpenAI classification
- **If 80-89% accuracy**: Consider hybrid approach (OpenAI + clustering backup)
- **If < 80% accuracy**: Continue with clustering approach

### Next Steps:
Based on the test results above, implement the appropriate approach in production.